In [ ]:
import os
import optuna
import torch
import numpy as np
from darts import TimeSeries
from darts.models import NBEATSModel
from darts.metrics import mae, rmse
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm
from darts.metrics import mae, rmse, mse, mape, smape
from sklearn.preprocessing import MinMaxScaler

prediction_length = prediction_length

def bootstrap_metrics(actual, predicted, n_bootstraps=1000, alpha=0.05, random_state=42):

    rng = np.random.default_rng(random_state)

    actual = np.asarray(actual).flatten()
    predicted = np.asarray(predicted).flatten()
    n = len(actual)

    actual_ts = TimeSeries.from_values(actual)
    pred_ts = TimeSeries.from_values(predicted)

    point_mae = mae(actual_ts, pred_ts)
    point_rmse = rmse(actual_ts, pred_ts)
    point_mse = mse(actual_ts, pred_ts)
    point_mape = mape(actual_ts, pred_ts)
    point_smape = smape(actual_ts, pred_ts)

    metrics_boot = {
        "mae": np.empty(n_bootstraps),
        "rmse": np.empty(n_bootstraps),
        "mse": np.empty(n_bootstraps),
        "mape": np.empty(n_bootstraps),
        "smape": np.empty(n_bootstraps),
    }

    for i in range(n_bootstraps):
        idx = rng.integers(0, n, n)
        y_true = actual[idx]
        y_pred = predicted[idx]

        a_ts = TimeSeries.from_values(y_true)
        p_ts = TimeSeries.from_values(y_pred)

        metrics_boot["mae"][i] = mae(a_ts, p_ts)
        metrics_boot["rmse"][i] = rmse(a_ts, p_ts)
        metrics_boot["mse"][i] = mse(a_ts, p_ts)
        metrics_boot["mape"][i] = mape(a_ts, p_ts)
        metrics_boot["smape"][i] = smape(a_ts, p_ts)

    lower_q, upper_q = 100 * (alpha / 2), 100 * (1 - alpha / 2)
    result = {}
    for name, arr in metrics_boot.items():
        low, high = np.percentile(arr, [lower_q, upper_q])
        result[name] = eval(f"point_{name}")
        result[f"{name}_ci_lower"] = low
        result[f"{name}_ci_upper"] = high

    return result


def get_train_test_split(series_list, prediction_length):
    # Splits each series into training and test part
    train_series = [s[:-prediction_length] for s in series_list]
    test_series = [s[-prediction_length:] for s in series_list]
    return train_series, test_series


def inner_objective(trial, train_series_list):
    # using inner 3-fold CV for HOP
    input_chunk_length = trial.suggest_int("input_chunk_length", 3, 9)
    layer_widths_value = trial.suggest_int("layer_widths_value", 128, 512)
    num_blocks = trial.suggest_int("num_blocks", 1, 6)
    num_stacks = trial.suggest_int("num_stacks", 1, 6)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)

    kf_inner = KFold(n_splits=3, shuffle=True, random_state=42)
    inner_cv_scores = []

    for train_idx, val_idx in kf_inner.split(train_series_list):
        fold_train_series = [train_series_list[i] for i in train_idx]
        fold_val_series = [train_series_list[i] for i in val_idx]

        # Scale per fold
        scaler = MinMaxScaler(feature_range=(0, 1))
        train_concat = np.concatenate([s.values() for s in fold_train_series])
        scaler.fit(train_concat)
        fold_train_scaled = [TimeSeries.from_values(scaler.transform(s.values())) for s in fold_train_series]
        fold_val_scaled = [TimeSeries.from_values(scaler.transform(s.values())) for s in fold_val_series]

        
        model = NBEATSModel(
            input_chunk_length=input_chunk_length,
            output_chunk_length=prediction_length,
            layer_widths=layer_widths_value,
            num_blocks=num_blocks,
            num_stacks=num_stacks,
            dropout=dropout,
            random_state=42,
            batch_size=batch_size,
            optimizer_kwargs={"lr": learning_rate},
            likelihood=None,
            pl_trainer_kwargs={
                "accelerator": "cpu",
                "devices": 1,
                "callbacks": [EarlyStopping(monitor="train_loss", patience=10, mode="min")]
            },
        )

        model.fit(series=fold_train_scaled, epochs=50, verbose=False)

        # Forecast and inverse-transform
        predictions_scaled = model.predict(n=prediction_length, series=fold_val_scaled)

        actual_values = np.concatenate([s.values().flatten() for s in fold_val_series])
        pred_values = np.concatenate([p.all_values().flatten() for p in predictions_scaled])

        actual_rescaled = scaler.inverse_transform(actual_values.reshape(-1, 1)).flatten()
        predicted_rescaled = scaler.inverse_transform(pred_values.reshape(-1, 1)).flatten()

        fold_mae = mae(TimeSeries.from_values(actual_rescaled), TimeSeries.from_values(predicted_rescaled))
        inner_cv_scores.append(fold_mae)

    return np.mean(inner_cv_scores)


def nested_cross_validation(full_series_dict, n_outer_folds=10, n_trials=50, max_epochs=500):
    all_series_list = list(full_series_dict.values())
    train_input_series, test_target_series = get_train_test_split(all_series_list, prediction_length)

    kf_outer = KFold(n_splits=n_outer_folds, shuffle=True, random_state=42)
    aggregated_actuals = []
    aggregated_predictions = []

    print(f"Starting Nested {n_outer_folds}-Fold Cross-Validation...")

    for fold_num, (train_index, test_index) in enumerate(kf_outer.split(train_input_series)):
        print(f"\n--- Outer Fold {fold_num+1}/{n_outer_folds} ---")

        # Define outer splits
        outer_train_series = [train_input_series[i] for i in train_index]
        outer_test_series = [train_input_series[i] for i in test_index]
        outer_test_targets = [test_target_series[i] for i in test_index]

        # Fit global scaler on outer-train
        all_train_values = np.concatenate([s.values().flatten() for s in outer_train_series])
        global_scaler = MinMaxScaler(feature_range=(0, 1))
        global_scaler.fit(all_train_values.reshape(-1, 1))

        # Transform
        outer_train_scaled = [TimeSeries.from_values(global_scaler.transform(s.values())) for s in outer_train_series]
        outer_test_scaled = [TimeSeries.from_values(global_scaler.transform(s.values())) for s in outer_test_series]
        outer_test_targets_scaled = [TimeSeries.from_values(global_scaler.transform(s.values())) for s in outer_test_targets]

        # Inner optimization
        study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
        wrapped_objective = lambda trial: inner_objective(trial, outer_train_scaled)
        study.optimize(wrapped_objective, n_trials=n_trials, show_progress_bar=False)
        best_params = study.best_trial.params
        print(f"Best hyperparameters: {best_params}")

        # Train
        best_model = NBEATSModel(
            input_chunk_length=best_params["input_chunk_length"],
            output_chunk_length=prediction_length,
            random_state=42,
            dropout=best_params["dropout"],
            num_blocks=best_params["num_blocks"],
            num_stacks=best_params["num_stacks"],
            batch_size=best_params["batch_size"],
            layer_widths=best_params["layer_widths_value"],
            optimizer_kwargs={"lr": best_params["learning_rate"]},
            likelihood=None,
            pl_trainer_kwargs={
                "accelerator": "cpu",
                "devices": 1,
                "callbacks": [EarlyStopping(monitor="train_loss", patience=20, mode="min")]
            },
        )

        best_model.fit(series=outer_train_scaled, epochs=max_epochs, verbose=False)

        # Forecast
        predictions_scaled = best_model.predict(n=prediction_length, series=outer_test_scaled)

        # Inverse-transform predictions
        actuals_fold, preds_fold = [], []
        for pred_ts, true_ts in zip(predictions_scaled, outer_test_targets_scaled):
            y_pred_scaled = pred_ts.values().flatten()[0]
            y_true_scaled = true_ts.values().flatten()[0]
            y_pred = global_scaler.inverse_transform([[y_pred_scaled]])[0, 0]
            y_true = global_scaler.inverse_transform([[y_true_scaled]])[0, 0]
            preds_fold.append(y_pred)
            actuals_fold.append(y_true)

        print("Example rescaled pairs (first 5):")
        for a, p in list(zip(actuals_fold, preds_fold))[:5]:
            print(f"True={a:.2f}, Pred={p:.2f}, Diff={abs(a-p):.2f}")

        aggregated_actuals.extend(actuals_fold)
        aggregated_predictions.extend(preds_fold)

    print("\nNested CV Complete. Running Bootstrapping")

    final_metrics = bootstrap_metrics(
        np.array(aggregated_actuals),
        np.array(aggregated_predictions),
        n_bootstraps=1000
    )

    return final_metrics

if __name__ == "__main__":
    if 'valid_series_dict_full' not in locals() and 'valid_series_dict_full' not in globals():
        print("Using dummy data for demonstration")
        valid_series_dict_full = {
            f"Encounter/{i}": TimeSeries.from_values(np.random.rand(15, 1))
            for i in range(100)
        }

    results = nested_cross_validation(
        full_series_dict=valid_series_dict_full,
        n_outer_folds=10,
        n_trials=50, 
        max_epochs=100
    )

    print(f"Aggregated Test-Fold Results")
    print(f"MAE:   {results['mae']:.3f}  (95% CI: [{results['mae_ci_lower']:.3f}, {results['mae_ci_upper']:.3f}])")
    print(f"RMSE:  {results['rmse']:.3f} (95% CI: [{results['rmse_ci_lower']:.3f}, {results['rmse_ci_upper']:.3f}])")
    print(f"MSE:   {results['mse']:.3f} (95% CI: [{results['mse_ci_lower']:.3f}, {results['mse_ci_upper']:.3f}])")
    print(f"MAPE:  {results['mape']:.2f}% (95% CI: [{results['mape_ci_lower']:.2f}%, {results['mape_ci_upper']:.2f}%])")
    print(f"sMAPE: {results['smape']:.2f}% (95% CI: [{results['smape_ci_lower']:.2f}%, {results['smape_ci_upper']:.2f}%])")

